In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import optuna

In [2]:
# make the result reproducible
keras.utils.set_random_seed(42)

In [3]:
physical_devices = tf.config.list_physical_devices()
print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

In [5]:
from utils.constrains import CLASSES, IMAGE_SIZE

BATCH_SIZE = 200
EPOCHS = 60

In [6]:
from keras import layers, regularizers, Sequential, Input
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy

def define_model(trial):

  conv_regularizer = regularizers.l2(l=trial.suggest_float("conv_regularizer", 0.0001, 0.001)) # 0.0008
  dense_regularizer = regularizers.l2(l=trial.suggest_float("dense_regularizer", 0.005, 0.05)) # 0.01

  dropout_base =trial.suggest_float("dropout_base", 0, 0.4)
  dropout_increment = trial.suggest_float("dropout_increment", 0, 0.25)
  activation_def = 'tanh' # trial.suggest_categorical("activation_def", ["tanh", "relu"])
  
  model = Sequential(
    [
      Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
      layers.Conv2D(6, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(dropout_base),
      layers.Conv2D(16, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      layers.MaxPooling2D(pool_size=2, strides=2),
      
      layers.Dropout(dropout_base + 1*dropout_increment),
      layers.Conv2D(120, kernel_size=5, padding='same', activation=activation_def, kernel_regularizer=conv_regularizer),
      
      layers.Flatten(),
      layers.Dropout(dropout_base + 2*dropout_increment),
      layers.Dense(84, activation=activation_def, kernel_regularizer=dense_regularizer),
      layers.Dense(CLASSES, activation='softmax'),
    ]
  )
  
  optimizer = Adam(learning_rate=0.001)

  model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=["accuracy"])
  
  return model

In [7]:
# Defines training and evaluation.
def train_model(model, X_train, y_train):
  
  history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    shuffle=True,
    verbose=0,
  )
  
  return history

def evaluate_trial(history):
  MEASUREMENT_SPAN = 5
  length = len(history.history['accuracy'])
  
  acc = np.mean(history.history['accuracy'][length-MEASUREMENT_SPAN:])
  val_acc = np.mean(history.history['val_accuracy'][length-MEASUREMENT_SPAN:])

  # Should be minimized
  difference = acc - val_acc
  
  return val_acc, difference

In [8]:
from utils.load_dataset import load_dataset

DIRPATH = './data'
X_train, X_test, y_train, y_test = load_dataset(DIRPATH)

def objective(trial):  
  model = define_model(trial)
  
  history = train_model(model, X_train, y_train)
  val_acc, difference = evaluate_trial(history)
  return val_acc, difference

In [9]:
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=30, timeout=300)

print("Number of finished trials: ", len(study.trials))

[I 2024-05-13 17:10:36,013] A new study created in memory with name: no-name-35b0556f-df25-4e7d-9054-eff16d389730
[I 2024-05-13 17:10:54,029] Trial 0 finished with values: [0.7396709322929382, -0.021838283538818337] and parameters: {'conv_regularizer': 0.00028613476045069087, 'dense_regularizer': 0.0394008739253034, 'dropout_base': 0.2722574395655608, 'dropout_increment': 0.09478758542340662}. 
[I 2024-05-13 17:11:07,143] Trial 1 finished with values: [0.7716636061668396, 0.11096098423004153] and parameters: {'conv_regularizer': 0.00028052644960810895, 'dense_regularizer': 0.03011617015950931, 'dropout_base': 0.03421660063882932, 'dropout_increment': 0.06876648844401273}. 
[I 2024-05-13 17:11:19,998] Trial 2 finished with values: [0.7840950608253479, 0.028250622749328658] and parameters: {'conv_regularizer': 0.0008899988456323993, 'dense_regularizer': 0.018699125115052097, 'dropout_base': 0.14105682038169562, 'dropout_increment': 0.14579671662781815}. 
[I 2024-05-13 17:11:33,038] Trial

Number of finished trials:  23
